# Data Processing and splitting

In [2]:
from logic.processing import load_data, preproc, balance_dataset, data_filter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report
from sklearn.pipeline import make_pipeline
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.callbacks import EarlyStopping

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Flotchi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/Flotchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/Flotchi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
2024-12-03 16:59:30.062772: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df_train = load_data('drugsComTrain_raw.csv')
df_test = load_data('drugsComTest_raw.csv')

In [4]:
df_train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [5]:
df_train_filter = data_filter(df_train)
df_test_filter = data_filter(df_test)

In [6]:
df_test_filter.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount,sentiment,review_length
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22,1,68
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17,1,48
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35,1,143
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4,1,149
7,169852,Amitriptyline,Migraine Prevention,"""This has been great for me. I&#039;ve been on...",9,21-Apr-09,32,1,64


In [7]:
df_train_prep = preproc(df_train_filter)
df_test_prep = preproc(df_test_filter)

In [8]:
X_train = df_train_prep['clean']
y_train = df_train_prep['sentiment']

In [9]:
X_test = df_test_prep['clean']
y_test = df_test_prep['sentiment']

In [10]:
X_train_b,y_train_b = balance_dataset(X_train,y_train)
X_test_b,y_test_b = balance_dataset(X_test,y_test)

In [11]:
len(y_train_b[y_train_b==0])==len(y_train_b[y_train_b==1])

True

# Tokenize

In [12]:
X_train_tk = [text_to_word_sequence(_) for _ in X_train_b]

In [13]:
X_test_tk = [text_to_word_sequence(_) for _ in X_test_b]

# Embedding

In [14]:
word2vec_model = Word2Vec(
    sentences=X_train_tk,
    vector_size=200,  # dimension des embeddings
    window=5,         # fenêtre de contexte
    min_count=5,      # mots minimums pour inclusion
    sg=1,             # Skip-Gram
    epochs=20,        # Nombre d'époques
)

In [16]:
len(word2vec_model.wv.key_to_index)

9900

In [17]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [18]:
X_train_embed = embedding(word2vec_model, X_train_tk)
X_test_embed = embedding(word2vec_model, X_test_tk)

# Padding

In [19]:
X_train_pad = pad_sequences(
    X_train_embed,
    maxlen=200,        # Longueur maximale de mots
    padding='pre',   # Padding avant la séquence
    truncating='post' # Tronquage à la fin
)

In [21]:
X_test_pad = pad_sequences(
    X_test_embed,
    maxlen=200,        # Longueur maximale de mots
    padding='pre',   # Padding avant la séquence
    truncating='post' # Tronquage à la fin
)

In [22]:
X_train_pad.shape,X_test_pad.shape

((56314, 200, 200), (19092, 200, 200))

# Model Building

In [111]:
def init_model():
    model = Sequential()
    model.add(InputLayer((200,200)))
    model.add(layers.Masking(mask_value=0.))
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [112]:
model = init_model()

In [113]:
model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

In [114]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_7 (Masking)             │ (None, 200, 60)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 20)             │         6,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 15)             │           315 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,811 (26.61 KB)

 Trainable params: 6,811 (26.61 KB)

 Non-trainable params: 0 (0.00 B)

# Model training

In [119]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train_b, 
          batch_size = 64,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 81s 124ms/step - accuracy: 0.6320 - loss: 0.6253 - val_accuracy: 0.7747 - val_loss: 0.4816
Epoch 2/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 57s 84ms/step - accuracy: 0.8012 - loss: 0.4438 - val_accuracy: 0.8224 - val_loss: 0.4115
Epoch 3/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 51s 82ms/step - accuracy: 0.8312 - loss: 0.3922 - val_accuracy: 0.8418 - val_loss: 0.3784
Epoch 4/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 51s 83ms/step - accuracy: 0.8521 - loss: 0.3509 - val_accuracy: 0.8459 - val_loss: 0.3648
Epoch 5/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 50s 82ms/step - accuracy: 0.8613 - loss: 0.3311 - val_accuracy: 0.8497 - val_loss: 0.3587
Epoch 6/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 66s 107ms/step - accuracy: 0.8737 - loss: 0.3103 - val_accuracy: 0.8400 - val_loss: 0.3808
Epoch 7/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 53s 85ms/step - accuracy: 0.8789 - loss: 0.2983 - val_accuracy: 0.8598 - val_loss: 0.3385
Epoch 8/100
616/616 ━━━━━━━━━━━━━━━━━━━━ 54s 87ms/step - accuracy: 0.8833 - loss:

# Evaluation

In [121]:
res = model.evaluate(X_test_pad, y_test_b, verbose=0)

In [122]:
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 87.256%


In [124]:
y_pred = model.predict(X_test_pad)

597/597 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step


In [140]:
y_pred = np.round(y_pred)

In [152]:
print(classification_report(y_test_b, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87      9546
           1       0.88      0.86      0.87      9546

    accuracy                           0.87     19092
   macro avg       0.87      0.87      0.87     19092
weighted avg       0.87      0.87      0.87     19092

